In [110]:
!pip install -U langchain-ollama langchain-openai

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [113]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [114]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [115]:
df.shape

(40000, 2)

In [116]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [117]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [118]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [119]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [120]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [121]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

CPU times: user 1.87 s, sys: 16.1 ms, total: 1.88 s
Wall time: 1.9 s


In [122]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [123]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.12 s, sys: 59.6 ms, total: 2.18 s
Wall time: 2.24 s


In [124]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [125]:
coffee_reviews_df.shape

(4981, 2)

In [126]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [127]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [128]:
!ollama pull gemma3:270m

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it


In [129]:
!ollama list

NAME                                  ID              SIZE      MODIFIED          
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    About an hour ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    About an hour ago    


In [130]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ 😊



In [131]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"scb10x/typhoon2.1-gemma3-4b:latest","model":"scb10x/typhoon2.1-gemma3-4b:latest","modified_at":"2025-10-19T13:50:14.967221405Z","size":2615662352,"digest":"8cfab9097c9d142fa16c7556826c9a7874f4642ac6d575e2d66a4f4979c48b76","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.9B","quantization_level":"Q4_K_M"}},{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T13:47:58.938377874Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [132]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [133]:
llm = OllamaLLM(model="gemma3:270m")

In [134]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊



In [135]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [136]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [137]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [138]:
chain = prompt | llm

In [139]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [140]:
response

'แน่นอนครับ! ผมจะช่วยคุณเขียนโค้ด Python เพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ `math.sum()`\n\n```python\nimport math\n\n# ตัวอย่างการใช้\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n\ntotal = 0\nfor number in numbers:\n    total += number\n\nprint("ผลรวม:", total)\n```\n\nโค้ดนี้จะทำงานโดยอาศัยการใช้ `math.sum()` เพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยให้ตัวแปร `total` เป็นจำนวนเต็มที่กำหนด และ `number` เป็นตัวเลขที่ต้องการหาผลรวม\n\n**คำอธิบายโค้ด:**\n\n1.  **`import math`:**  ใช้ไลบริสต์ `math` เพื่อเข้าถึงและใช้ฟังก์ชัน `math.sum()`\n2.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`:**  กำหนดตัวแปร `numbers` ที่มีค่าเป็นจำนวนเต็ม 1 ถึง 10\n3.  **`total = 0`:**  กำหนดค่าเริ่มต้น `total` เป็น 0\n4.  **`for number in numbers:`:**  วนซ้ำผ่านตัวแปร `numbers` โดยใช้ `for` loop\n5.  **`total += number`:**  ทำการเพิ่มค่าของ `total` ไปยังตัวแปร `number`  โดยใช้ `+=`\n6.  **`print("ผลรวม:", total)`:**  แสดงผลลัพธ์ของโค้ด\n\n**ตัวอย่างการใช้งาน:**\n\n```\nผลรวม: 15\n```\n\n**คำแนะนำเพิ่มเติม:**\n\n

In [141]:
from IPython.display import display, Markdown

In [142]:
display(Markdown(response))

แน่นอนครับ! ผมจะช่วยคุณเขียนโค้ด Python เพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยใช้ `math.sum()`

```python
import math

# ตัวอย่างการใช้
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

total = 0
for number in numbers:
    total += number

print("ผลรวม:", total)
```

โค้ดนี้จะทำงานโดยอาศัยการใช้ `math.sum()` เพื่อหาผลรวมของตัวเลข 1 ถึง 10 โดยให้ตัวแปร `total` เป็นจำนวนเต็มที่กำหนด และ `number` เป็นตัวเลขที่ต้องการหาผลรวม

**คำอธิบายโค้ด:**

1.  **`import math`:**  ใช้ไลบริสต์ `math` เพื่อเข้าถึงและใช้ฟังก์ชัน `math.sum()`
2.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`:**  กำหนดตัวแปร `numbers` ที่มีค่าเป็นจำนวนเต็ม 1 ถึง 10
3.  **`total = 0`:**  กำหนดค่าเริ่มต้น `total` เป็น 0
4.  **`for number in numbers:`:**  วนซ้ำผ่านตัวแปร `numbers` โดยใช้ `for` loop
5.  **`total += number`:**  ทำการเพิ่มค่าของ `total` ไปยังตัวแปร `number`  โดยใช้ `+=`
6.  **`print("ผลรวม:", total)`:**  แสดงผลลัพธ์ของโค้ด

**ตัวอย่างการใช้งาน:**

```
ผลรวม: 15
```

**คำแนะนำเพิ่มเติม:**

*   **การใช้ `math.sum()`:**  การใช้ `math.sum()`  เป็นวิธีที่ง่ายและเป็นมาตรฐานในการหาผลรวมของตัวเลขใน Python
*   **การปรับแต่งตัวแปร:**  คุณสามารถปรับแต่งตัวแปร `total` ได้โดยใช้ `range()` เพื่อสร้างตัวแปรที่เปลี่ยนแปลงไปตามจำนวนตัวเลขที่ต้องการ
*   **การจัดการกับค่าคงที่:**  หากคุณต้องการจัดการกับค่าคงที่ (เช่น ค่าที่ต้องใช้ในการคำนวณ)  คุณสามารถใช้ `try...except` เพื่อจัดการกับข้อผิดพลาด

ถ้าคุณต้องการลองเขียนโค้ดอื่น ๆ ที่มีตัวแปรที่แตกต่างกัน หรือต้องการใช้ `math.sum()` ในบริบทอื่น ๆ โปรดแจ้งให้ทราบนะครับ ผมยินดีช่วยเหลือ!

In [143]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [144]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [145]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [146]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [147]:
parser_chain = StrOutputParser()

In [148]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [149]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [150]:
result

'วิธิีทำจากผลลัทธ์ มีลักษณะเด่นคือ **การสร้างและควบคุมระบบการบริหารจัดการ** ซึ่งเป็นส่วนสำคัญของการดำเนินงานและประสิทธิภาพขององค์กร'

In [151]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON โดยมี key คือ keywords เช่น {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [152]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [153]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [154]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [155]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 38.7 ms, sys: 2.51 ms, total: 41.2 ms
Wall time: 5.1 s


In [156]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": [
    "ร้านอาหารใหญ่",
    "ห้องน้ำ",
    "กาแฟ",
    "น้ำผึ้ง",
    "น้ำผึ้งมาราด",
    "แพงเวอร์",
    "ไม่ประทับใจ",
    "ลาบไข่ต้ม",
    "ไข่มันคาว",
    "ขนมไทย",
    "ขนมหวานฟรี",
    "ขนมไทย",
    "ขนมไทย"
  ]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "รสชาติ",
    "กลิ่น",
    "เมพพ",
    "ซากอารยธรรมลาเต้",
    "ซอยไรจำไม่ได้",
    "แต่อยู่ตรงข้าม Villa",
    "BS อารีย์"
  ]
}
```



In [157]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [158]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    About an hour ago         


In [159]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b:latest")

In [160]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [161]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 101 ms, sys: 11 ms, total: 112 ms
Wall time: 17.5 s


In [162]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะช่วยคุณวิเคราะห์และดึงคำสำคัญจากรีวิวนี้ออกมาเป็น JSON ตามขั้นตอนที่คุณต้องการ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้เป็นความคิดเห็นเกี่ยวกับร้านอาหารแห่งหนึ่ง โดยมีการกล่าวถึงขนาดของร้าน, อาหารบางอย่างที่สั่ง, ราคา, และประสบการณ์โดยรวม

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้าน
*   อาหาร (ลาบไข่ต้ม, ทอดมันหัวปลี, พะแนงห่อไข่, ขนมหวาน)
*   ราคา

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อหลักที่กล่าวถึง ผมเลือกคำเหล่านี้เป็น keywords:

1.  ราคา (เนื่องจากมีการกล่าวถึงราคาอาหารหลายครั้งและเป็นประเด็นสำคัญที่ผู้รีวิวไม่พอใจ)
2.  อาหาร (ครอบคลุมอาหารหลากหลายประเภทที่กล่าวถึง)
3.  ขนาดร้าน (เพื่อบ่งบอกถึงความใหญ่ของร้าน)

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "อาหาร", "ขนาดร้าน"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากคุณต้องการให้ผมช่วยอะไรเพิ่มเติม บอกได้เลยครับ

Review วันนี้ได้มีโอกาสไป

In [163]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [164]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [165]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [166]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 126 ms, sys: 7.05 ms, total: 133 ms
Wall time: 12 s


In [167]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'ขนาดร้าน']

In [168]:
results[1]['text']['keywords']

['วาวี', 'ลาเต้', 'บรรยากาศ']

In [169]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [170]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 121 ms, sys: 10.6 ms, total: 132 ms
Wall time: 12 s


In [171]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'วาวี'])

In [172]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [173]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [174]:
inputs = [{"review": r} for r in sample_reviews]

In [175]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 21.7 ms, sys: 1.51 ms, total: 23.2 ms
Wall time: 2.95 s


In [176]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [177]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [178]:
from tqdm import tqdm

In [179]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [19:26<00:00,  9.33s/it]


In [180]:
for result in results_1000[0:10]:

    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [181]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [182]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [183]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [184]:
context_extract = context_prompt | llm3 | context_parser

In [185]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [186]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [187]:
results_1000[0]['text']['keywords'].split(',')

['ราคา', ' บรรยากาศ', ' ไข่ต้ม']

In [188]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 142078.66it/s]


In [189]:
len(context_inputs)

3000

In [190]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list


Extracting contexts: 100%|██████████| 1000/1000 [1:12:54<00:00,  4.37s/it]


In [191]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้

In [192]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 297405.09it/s]


In [193]:
len(sentiment_inputs)

3000

In [194]:
sentiment_inputs[:6]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'บรรยากาศร้านใหญ่มากกกกกกก'},
 {'message': 'ไข่ต้ม'},
 {'message': 'กาแฟที่นี่อร่อยมากกกกกกก เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'}]

In [196]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 1000/1000 [42:45<00:00,  2.57s/it]


In [197]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม'],
  'sentiment': ['negative', 'positive', 'neutral']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล

In [198]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'].split(','))},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [199]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')